# How to simply use keras
* Reference
    + https://www.tensorflow.org/guide/keras

### Setup

In [1]:
import numpy as np
import tensorflow as tf

keras = tf.keras

print(tf.__version__)
print(tf.keras.__version__)

1.12.0
2.1.6-tf


### Build a simple model
#### Sequential model
In Keras, you assemble layers to build models. A model is (usually) a graph of layers. The most common type of model is a stack of layers: the `tf.keras.Sequential` model.

In [2]:
## To build a simple, fully-connected network (i.e. multi-layer perceptron)
# If you specify the input shape, the model gets built continuously, as you are adding layers.
# Note that when using this delayed-build pattern (no input shape specified),
# the model doesn't have any weights until the first call,
# to a training/evaluation method (since it isn't yet built)

model = keras.Sequential()
model.add(keras.layers.Dense(units = 64, activation = 'relu', input_shape = (16,))) 
# or model.add(keras.layers.Dense(units = 64, activation = 'relu'))
model.add(keras.layers.Dense(units = 64, activation = 'relu'))
model.add(keras.layers.Dense(units = 10, activation = 'softmax'))

#### Configure the layers
There are many tf.keras.layers available with some common constructor parameters:

* `activation`: Set the activation function for the layer. This parameter is specified by the name of a built-in function or as a callable object. By default, no activation is applied.
* `kernel_initializer` and `bias_initializer`: The initialization schemes that create the layer's weights (kernel and bias). This parameter is a name or a callable object. This defaults to the `"Glorot uniform"` initializer.
* `kernel_regularizer` and `bias_regularizer`: The regularization schemes that apply the layer's weights (kernel and bias), such as L1 or L2 regularization. By default, no regularization is applied.  
  
The following instantiates `tf.keras.layers.Dense` layers using constructor arguments:

In [3]:
tf.reset_default_graph()

# Create a sigmoid layer:
keras.layers.Dense(64, activation='sigmoid')
# Or:
keras.layers.Dense(64, activation=tf.sigmoid)

# A linear layer with L1 regularization of factor 0.01 applied to the kernel matrix:
keras.layers.Dense(64, kernel_regularizer=tf.keras.regularizers.l1(0.01))

# A linear layer with L2 regularization of factor 0.01 applied to the bias vector:
keras.layers.Dense(64, bias_regularizer=tf.keras.regularizers.l2(0.01))

# A linear layer with a kernel initialized to a random orthogonal matrix:
keras.layers.Dense(64, kernel_initializer='orthogonal')

# A linear layer with a bias vector initialized to 2.0s:
keras.layers.Dense(64, bias_initializer=tf.keras.initializers.constant(2.0))

### Train and evaluate

#### Set up training
After the model is constructed, configure its learning process by calling the `compile` method, `tf.keras.Model.compile` or `tf.keras.Sequential.compile` takes three important arguments.  
  
* `optimizer`: This object specifies the training procedure. Pass it optimizer instances from the `tf.train` module, such as `tf.train.AdamOptimizer`, `tf.train.RMSPropOptimizer`, or `tf.train.GradientDescentOptimizer`.
* `loss`: The function to minimize during optimization. Common choices include mean square error (`mse`), `categorical_crossentropy`, and `binary_crossentropy`. Loss functions are specified by name or by passing a callable object from the `tf.keras.losses` module.
* `metrics`: Used to monitor training. These are string names or callables from the `tf.keras.metrics` module.

In [4]:
tf.reset_default_graph()

model = keras.Sequential()
model.add(keras.layers.Dense(units=64, activation='relu'))
model.add(keras.layers.Dense(units=64, activation='relu'))
model.add(keras.layers.Dense(units=10, activation='softmax'))

model.compile(optimizer=tf.train.AdamOptimizer(0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

The following shows a few examples of configuring a model for training:

In [5]:
# Configure a model for mean-squared error regression.
model.compile(optimizer=tf.train.AdamOptimizer(0.01),
              loss='mse',       # mean squared error
              metrics=['mae'])  # mean absolute error

# Configure a model for categorical classification.
model.compile(optimizer=tf.train.RMSPropOptimizer(0.01),
              loss=tf.keras.losses.categorical_crossentropy,
              metrics=[tf.keras.metrics.categorical_accuracy])

#### Input NumPy data

In [6]:
# Numpy dataset
tf.reset_default_graph()
tr_data = np.random.random((1000, 32))
tr_labels = np.random.randint(low=0, high=10, size = 1000)

val_data = np.random.random((100, 32))
val_labels = np.random.randint(low=0, high=10, size = 100)

tst_data = np.random.random((100, 32))
tst_labels = np.random.randint(low=0, high=10, size = 100)

In [8]:
# Training
model = keras.Sequential()
model.add(keras.layers.Dense(units=64, activation='relu', input_shape = (32,)))
model.add(keras.layers.Dense(units=64, activation='relu'))
model.add(keras.layers.Dense(units=10, activation='softmax'))
model.compile(optimizer=tf.train.GradientDescentOptimizer(.01), 
              loss=keras.losses.sparse_categorical_crossentropy,
              metrics=[keras.metrics.sparse_categorical_accuracy])

model.fit(x=tr_data, y=tr_labels, epochs=5, batch_size=32, validation_data=(val_data, val_labels))

Train on 1000 samples, validate on 100 samples
Epoch 1/5
1000/1000 [==============================] - 0s 155us/step - loss: 2.3395 - sparse_categorical_accuracy: 0.0860 - val_loss: 2.2991 - val_sparse_categorical_accuracy: 0.1800
Epoch 2/5
1000/1000 [==============================] - 0s 47us/step - loss: 2.3267 - sparse_categorical_accuracy: 0.0810 - val_loss: 2.3030 - val_sparse_categorical_accuracy: 0.1300
Epoch 3/5
1000/1000 [==============================] - 0s 36us/step - loss: 2.3199 - sparse_categorical_accuracy: 0.0870 - val_loss: 2.3058 - val_sparse_categorical_accuracy: 0.1200
Epoch 4/5
1000/1000 [==============================] - 0s 42us/step - loss: 2.3145 - sparse_categorical_accuracy: 0.0870 - val_loss: 2.3094 - val_sparse_categorical_accuracy: 0.1100
Epoch 5/5
1000/1000 [==============================] - 0s 42us/step - loss: 2.3117 - sparse_categorical_accuracy: 0.0840 - val_loss: 2.3114 - val_sparse_categorical_accuracy: 0.0800


In [9]:
# Evaluate and predict
print(model.metrics_names)
print(model.evaluate(x=tst_data, y=tst_labels))
print(model.predict(x=tst_data).shape)

['loss', 'sparse_categorical_accuracy']
100/100 [==============================] - 0s 39us/step
[2.303616523742676, 0.11]
(100, 10)


#### Input tf.data datasets
Pass a `tf.data.Dataset` instance to the `fit`, `evaluate`, `predict` method.

In [13]:
# Numpy dataset
tf.reset_default_graph()
tr_data = np.random.random((1000, 32))
tr_labels = np.random.randint(low=0, high=10, size = 1000)
tr_dataset = tf.data.Dataset.from_tensor_slices((tr_data, tr_labels))
# tr_dataset = tr_dataset.shuffle(buffer_size=1000)
tr_dataset = tr_dataset.batch(batch_size=32)
print(tr_dataset.output_types)

# val_data = np.random.random((100, 32))
# val_labels = np.random.randint(low=0, high=10, size = 100)
# val_dataset = tf.data.Dataset.from_tensor_slices((val_data, val_labels))
# val_dataset = val_dataset.batch(batch_size=32).repeat()

# tst_data = np.random.random((100, 32))
# tst_labels = np.random.randint(low=0, high=10, size = 100)
# tst_dataset = tf.data.Dataset.from_tensor_slices((tst_data, tst_labels))
# tst_dataset = tst_dataset.batch(batch_size=32)

(tf.float64, tf.int64)


In [14]:
# Training
model = keras.Sequential()
model.add(keras.layers.Dense(units=64, activation='relu'))
model.add(keras.layers.Dense(units=64, activation='relu'))
model.add(keras.layers.Dense(units=10, activation='softmax'))
model.compile(optimizer=tf.train.GradientDescentOptimizer(.01), 
              loss=keras.losses.sparse_categorical_crossentropy,
              metrics=[keras.metrics.sparse_categorical_accuracy])

In [16]:
model.fit(tr_dataset, epochs = 5,steps_per_epoch = 1000//32)

ValueError: Fetch argument <tf.Operation 'MakeIterator' type=MakeIterator> cannot be interpreted as a Tensor. (Operation name: "MakeIterator"
op: "MakeIterator"
input: "BatchDatasetV2"
input: "IteratorV2"
attr {
  key: "_class"
  value {
    list {
      s: "loc:@IteratorV2"
    }
  }
}
 is not an element of this graph.)